### Creating Input-Output Target Pairs

In [7]:
with open("../the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

**we need to enocde this raw text using bpe**
**we'll use tiktoken library to do so**

In [8]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Haseeb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")


**encoding raw text now**

In [11]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


**removing first 50 tokens as sample out of whole encoded text**

In [16]:
enc_sample = enc_text[50:]

In [21]:
tokenizer.decode(enc_sample[:2])

' and established'

**creating input-output pairs**

In [17]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:       {y}")

x: [290, 4920, 2241, 287]
y:       [4920, 2241, 287, 257]


**next word perdiction**

In [27]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    
    print(context, "--->", desired, "||", tokenizer.decode(context), "--->", tokenizer.decode([desired]))

[290] ---> 4920 ||  and --->  established
[290, 4920] ---> 2241 ||  and established --->  himself
[290, 4920, 2241] ---> 287 ||  and established himself --->  in
[290, 4920, 2241, 287] ---> 257 ||  and established himself in --->  a


**Next we use the above technique to our raw text, i.e. books data, but for this much huge data, we need a good memory and sometime a GPU. To avoid this computational resource defficiency, we can use the technique of parallel prcocessing.**

#### Using dataloader to divide data into batches for parallel processing
##### Using Pytorch

**Implementing DataLoader**

step 1: Tokenize the entire text

Step 2: Use a slidin window to chunk the book into overlappong sequence of max_length

Step 3: Return the total number of rows in the dataset

Step 4: Return a single row from the dataset

In [29]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Haseeb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [30]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride): 
        self.input_ids = []
        self.target_ids = []
        
        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        # Use a sliding wnidow to chunk the book into overlapping sequence of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]

Step 1: intialize the tokenizer

Step 2: create dataset

Step 3: drop_last=True drops the last batch if it is shorter than the specifie batch_size to prevent loss spikes during training

Step 4: The number of CPU process to user for preprocessing

In [31]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    # initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")
    
    # create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    # create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
    
    return dataloader

**Testing the dataloader with a batch size of 1 for an LLM with a context size of 4**

In [33]:
print(raw_text[:50])

I HAD always thought Jack Gisburn rather a cheap g


**convert dataloader into a Python iterator to fetch the next entry via python's built-in next() function**

In [ ]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [36]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [39]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
